In [2]:
filepath = "Caro-Interviews/"     #Input audio file path
output_filepath = "Caro-Transcrips-Audio-Text/" #Final transcript path
bucketname = "voztext" #Name of the bucket created in the step before
audio_file_name = "Aufnahme_Florian.wav"
#!pip install pydub
#!pip install google-cloud-storage

# Import libraries
from pydub import AudioSegment
import io
import os
from google.cloud import speech
import wave
from google.cloud import storage
import time

from google.cloud import storage
# WARNING; WORKAROUND to prevent timeout for files > 6 MB on 800 kbps upload link.
storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024* 1024  # 50 MB
storage.blob._MAX_MULTIPART_SIZE = 5 * 1024* 1024  # 50 MB

In [27]:
!ls Caro-Interviews/

'Aufnahme Berit.MP3'	'Aufnahme Franziska.MP3'  'Aufnahme Thomas.MP3'
'Aufnahme Eric.MP3'	'Aufnahme Leonie.MP3'	   test1.wav
'Aufnahme Florian.MP3'	'Aufnahme Maren 1.MP3'


In [3]:
#credenciales googles para copyrigth
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= 'automatic-rite-301117-93bb0ddbd62b.json'   #ensure the path is set correctly
!echo $GOOGLE_APPLICATION_CREDENTIALS

automatic-rite-301117-93bb0ddbd62b.json


In [4]:
#Transforma el archivo de audio a texto 
def mp3_to_wav(audio_file_name):
    if audio_file_name.split('.')[1] == 'mp3':    
        sound = AudioSegment.from_mp3(audio_file_name)
        audio_file_name = audio_file_name.split('.')[0] + '.wav'
        sound.export(audio_file_name, format="wav")

In [5]:
#Transforma el archivo de audio de stereo a mono
def frame_rate_channel(audio_file_name):
    file_name = os.path.join(os.path.dirname('Caro-Interviews/'), 'Aufnahme_Berit.wav')
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels

In [6]:
#Charge files into google cloud bucket
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

In [7]:
#Delete files from google cloud bucket

def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()

In [8]:
file_name = filepath + audio_file_name


In [114]:

file_name = filepath + audio_file_name
mp3_to_wav(file_name)

# The name of the audio file to transcribe
    
#frame_rate, channels = frame_rate_channel("Aufnahme_Berit.wav")
    
if channels > 1:
    frame_rate_channel("Aufnahme_Florian.wav")
    
bucket_name = bucketname
source_file_name = filepath + audio_file_name
destination_blob_name = audio_file_name
    
upload_blob(bucket_name, source_file_name, destination_blob_name)
    
gcs_uri = 'gs://' + bucketname + '/' + audio_file_name
transcript = ''
    
client = speech.SpeechClient()
audio = speech.RecognitionAudio(uri=gcs_uri)

config = speech.RecognitionConfig(
encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
sample_rate_hertz=frame_rate,
language_code='de-DE')

# Detects speech in the audio file
operation = client.long_running_recognize(config= config, audio= audio)
response = operation.result(timeout=10000000)

for result in response.results:
    transcript += result.alternatives[0].transcript
    #print(u"Transcript: {}".format(result.alternatives[0].transcript))
    #print("Confidence: {}".format(result.alternatives[0].confidence))
    
delete_blob(bucket_name, destination_blob_name)
    

In [115]:
transcript_filename="test8"
f= open(output_filepath + transcript_filename,"w+")
f.write(transcript)
f.close()

In [9]:
from google.cloud import speech_v1p1beta1 as speech

file_name = filepath + audio_file_name
#mp3_to_wav(file_name)

# The name of the audio file to transcribe
    
frame_rate, channels = frame_rate_channel(file_name)
if channels > 1:
        stereo_to_mono(file_name)
    
bucket_name = bucketname
source_file_name = filepath + audio_file_name
destination_blob_name = audio_file_name
    
upload_blob(bucket_name, source_file_name, destination_blob_name)
    
gcs_uri = 'gs://' + bucketname + '/' + audio_file_name
transcript = ''
    
client = speech.SpeechClient()
audio = speech.RecognitionAudio(uri=gcs_uri)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=frame_rate,
    language_code='de-DE',
    enable_speaker_diarization=True,
    diarization_speaker_count=2) #Changed

    # Detects speech in the audio file
operation = client.long_running_recognize(config = config, audio = audio)
response = operation.result(timeout=10000)
result = response.results[-1] #Changed
words_info = result.alternatives[0].words #Changed
    
tag=1 #Changed
speaker="" #Changed

for word_info in words_info: #Changed
    if word_info.speaker_tag==tag: #Changed
        speaker=speaker+" "+word_info.word #Changed
    else: #Changed
        transcript += "speaker {}: {}".format(tag,speaker) + '\n' #Changed
        tag=word_info.speaker_tag #Changed
        speaker=""+word_info.word #Changed
transcript += "speaker {}: {}".format(tag,speaker) #Changed
    
delete_blob(bucket_name, destination_blob_name)
    



In [10]:
transcript_filename="Transcript-Florian"
f= open(output_filepath + transcript_filename,"w+")
f.write(transcript)
f.close()